# 1.

Написать функцию `anagrams :: String -> [String] -> [[String]]`, которая:

- по заданной строке, состоящей из слов, разделенных пробелами (слова состоят только из строчных латинских букв)
- словарю (слова в словаре также состоят из строчных латинских букв и не содержат пробелов) 

находит все наборы слов из словаря (анаграммы исходной строки) такие, что каждый получившийся набор состоит из всех букв исходной строки. 

Например:

> ```haskell 
anagrams "madonna louise ciccone"["come","cool","dance","in","income","musician","nude","occasional","one","two"]

должен выдать список анаграмм

> ```haskell
[["income","nude","occasional"],["cool","dance","musician","one"],["come","in","nude","occasional"]]

Слова из словаря могут входить в анаграмму неоднократно. Конечно, список может оказаться и пустым. Функция должна работать за приемлемое время на тестах, подобных вышеприведенному примеру.

In [1]:
-- TASK1

--сортирую список кортежей по количеству символов

sortBy :: [(Char,Int)] -> [(Char,Int)]
sortBy [] = []
sortBy x = mix : sortBy xx
           where mix = foldl (\ a p -> if fst a > fst p then p else a) (head x) x
                 xx  = filter (/= mix) x
                 
-- сравниваю два списка кортежей между собой

compare (x:xs) (y:ys) = (x == y) && compare xs ys
compare [] [] = True
compare _ _ = False

-- считаю количество букв в строке
               
countChars :: String -> [(Char,Int)]
countChars ""     = []
countChars (s:ss) =  (s,n) : countChars ww   
                       where ww = filter (/= s) ss
                             n  = length ss-length ww+1
                             
subset :: [String] -> [[String]]
subset [x] = [[x], []]
subset (x:ls) = current ++ map (x:) current
            where current = subset ls               
 
-- проверяю на соответствие встречаемости символов в исходной строке string для множества слов, выбираю лишь совпадающие
anagrams :: String -> [String] ->[[String]]
anagrams string dict =  filter (isEqual inp_string) subsets     
                        where subsets = subset dict
                              inp_string = sortBy $ countChars (head (cutWhitespace [string]))
                              isEqual s1 s2 = compare s1 (sortBy $ countChars (concat s2)) 
                              cutWhitespace (x:ls) = filter (/=' ') x:ls

In [2]:
let input_string = "madonna louise ciccone"
let dictionary = ["come","cool","dance","in","income","musician","nude","occasional","one","two"]
anagrams input_string dictionary

[["income","nude","occasional"],["cool","dance","musician","one"],["come","in","nude","occasional"]]

# 2.

Написать функцию `operations :: Integer -> Maybe String`, которая пытается расставить знаки '+' и '-' между цифрами `1, 2, 3, 4, 5, 6, 7, 8, 9` так, чтобы получилось заданное число. 

Между цифрами может не оказаться никакого знака, тогда цифры объединяются в число.

Например, один из вариантов расстановки для числа 118 - это `123+45-67+8+9`. 

Если знаки расставить удается, то результат выдается в виде строки с конструктором Just, так что operations 118 может выдать строку `Just "123+45-67+8+9"` или Just `"1+2+34+5-6-7+89"`. 

Если расставить знаки подходящим образом невозможно, то функция должна выдавать `Nothing`, например, вызов функции operations `2013` выдаст `Nothing`.

In [3]:
import Data.List
import Data.List.Split


-- вспомогательная функция - разделения для дерева
splits :: Eq a => [a] -> [([a], [a])]
splits xs = [ (as,bs) | (as,bs) <- zip (inits xs) (tails xs), as /= [], bs /= [] ]


-- операции + - и слияние
type Operation = Char
oper :: [Operation]
oper = ['+', '-', '\0'] 



-- Наше дерево выражений, в котором будут храниться значения и операции между ними
data Node = Value Integer  -- лист дерева с некоторым числовым значением 
            | Op Operation Node Node -- узел дерева: операция и два поддерева
            deriving Show

-- генерация всевозможных последовательность из цифр 1..9, +,- и слияния
gen :: [Integer] -> [Node]
gen [] = []
gen [x] = [Value x]
gen xs = [ Op op leftNode rightNode | (leftList,rightList) <- splits xs, 
                 leftNode <- gen leftList, 
                 rightNode <- gen rightList,
                 op <- oper]

-- для визуализации ответа - собираем дерево 
mynode :: Node -> String
mynode (Value a) = show a
mynode (Op op left right) | op == '\0' = mynode left ++ "" ++ mynode right
                            | otherwise = mynode left ++ [op] ++ mynode right

-- дополнительная функция для подсчета result - по строке "1+2+3-45.." высчитать результат
result2 :: [String] -> Integer
result2 [] = 0
result2 (x:ls) | length (splitOn "-" x) /= 1 = -minus (splitOn "-" x) + result2 ls
              | otherwise = (read x :: Integer) + result2 ls

-- дополнительная функция для result - для минуса
minus :: [String] -> Integer
minus [] = 0
minus (x:ls) = - (read x) - (minus ls)

-- дополнительная функция для result - для плюса
plus :: [String] -> Integer
plus = foldr (\ x -> (+) (read x :: Integer)) 0

-- по строке "1+2+3-45.." высчитать результат
result :: String -> Integer
result x | x == "123456789" = 123456789
         | length (splitOn "-" x) == 1 = plus (splitOn "+" x)
         | length (splitOn "+" x) == 1 = (read (head (splitOn "-" x)) :: Integer) - plus (tail (splitOn "-" x))
         | otherwise = result2 (splitOn "+" x)

-- ищем по индексу ответ в сгененированном списке gen
findIndex p xs = case [ i | (x, i) <- zip (map result xs) [0..], p x ] of
    [] -> Nothing
    e:_ -> Just (xs !! e)

-- результат
operations :: Integer -> Maybe String
operations p = findIndex (== p) (map mynode (gen [1..9]))

Line 45: Use foldr
Found:
minus [] = 0
minus (x : ls) = - (read x) - (minus ls)
Why not:
minus ls = foldr (\ x -> (-) (- (read x))) 0 lsLine 46: Redundant bracket
Found:
- (read x) - (minus ls)
Why not:
- (read x) - minus lsLine 66: Use elemIndex
Found:
findIndex (== p)
Why not:
elemIndex p

In [4]:
operations 123

Just "1+2+34+5-6+78+9"

In [5]:
operations 30

Just "12+3+4-5+6-7+8+9"

In [6]:
operations 100

Just "1+23-4+56+7+8+9"